<a href="https://colab.research.google.com/github/JeanMusenga/ASSORT-Automatic-Summarization-of-Stack-Overflow-Posts/blob/main/TextRank_TextSummarization_WithEffectiveToknization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Important resources:

https://chatgpt.com/share/0fde4435-eb1d-4ab1-9149-78c0899251e0

https://chatgpt.com/share/0fde4435-eb1d-4ab1-9149-78c0899251e0

# Importing necessary libraries

In [ ]:
pip install contractions colorama

In [ ]:
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
import numpy as np
import pandas as pd
import contractions
import unicodedata
import chardet
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from nltk.corpus import stopwords
import re

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Preprocessing the Text
First, let's preprocess the text data in the Question_body and Answer_body columns.

In [ ]:
dataset=pd.read_excel('366_ARPs_for_extracting_Issue_Solution_Pairs.xlsx')

In [ ]:
#to check for missing values.
dataset['Answer_body_cleaned'].isnull().sum()

0

In [ ]:
# Initialize the WordNet Lemmatizer
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Sentence tokenization
    sentences = sent_tokenize(text)

    # Stopwords
    stop_words = set(ENGLISH_STOP_WORDS)

    # Define punctuation and special characters
    punctuation = set(string.punctuation)
    extra_special_characters = set(["''", '``', '##', '>>', '<<', 'e', 'g', 'eg', 'cant', 'cannot', 'isnt', 'would', 'could', 'doesnt', 'hasnt', 'thanks', '-', ')', '\n'])
    special_characters = extra_special_characters.union(punctuation)

    processed_sentences = []

    for sentence in sentences:
        # Tokenization and cleaning
        cleaned_words = re.findall(r'\b\w+\b', sentence.lower())
        cleaned_words = [word for word in cleaned_words if word not in stop_words]
        cleaned_words = [word for word in cleaned_words if word not in special_characters]
        cleaned_words = [word for word in cleaned_words if not word.isdigit()]
        cleaned_words = [word for word in cleaned_words if word.isalpha()]

        # Lemmatization
        lemmatized_words = [lemmatizer.lemmatize(word) for word in cleaned_words]

        processed_sentences.append(cleaned_words)

    return sentences, processed_sentences

In [ ]:
# Preprocess the Question_body and Answer_body columns
dataset['processed_question'] = dataset['Question_body_cleaned'].apply(preprocess_text)
dataset['processed_answer'] = dataset['Answer_body_cleaned'].apply(preprocess_text)

dataset[['processed_question', 'processed_answer']].head()

,processed_question,processed_answer
0,([I need help with the architecture pattern I ...,([So first of all what we're talking about her...
1,([which part of file structure we should do pr...,"([For the purpose of clean architecture, the f..."
2,([I am building c# .NET 4.8.1 MVC web applicat...,([There is no easy and reliable way to do this...
3,([We have a mobile application that we scale a...,([There is not a simple answer to this questio...
4,([I'm trying to properly design an application...,([Determining the source of the information is...


# Building the Similarity Matrix
Next, we built the similarity matrix for the preprocessed sentences using TF-IDF and calculate cosine similarities.

In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def build_similarity_matrix(processed_sentences):
    sentences = [" ".join(sentence) for sentence in processed_sentences]
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)
    similarity_matrix = cosine_similarity(tfidf_matrix)

    return similarity_matrix

# Build similarity matrices for both processed questions and answers
dataset['question_similarity_matrix'] = dataset['processed_question'].apply(lambda x: build_similarity_matrix(x[1]))
dataset['answer_similarity_matrix'] = dataset['processed_answer'].apply(lambda x: build_similarity_matrix(x[1]))

dataset[['question_similarity_matrix', 'answer_similarity_matrix']].head()


,question_similarity_matrix,answer_similarity_matrix
0,"[[1.0, 0.0, 0.09473376964436592, 0.0, 0.0, 0.0...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1456608..."
1,"[[1.0000000000000002, 0.0, 0.0, 0.0, 0.0, 0.0]...","[[1.0, 0.0, 0.0865651075502646, 0.034802321376..."
2,"[[1.0, 0.07028081026664326, 0.0690588812162663...","[[0.9999999999999999, 0.17557494911455973, 0.1..."
3,"[[1.0000000000000002, 0.0, 0.11193188131429124...","[[1.0000000000000002, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,"[[0.9999999999999999, 0.097745674227839, 0.033...","[[1.0, 0.11062087717781531, 0.1267026034121509..."


# Applying TextRank Algorithm

In [ ]:
import numpy as np

def text_rank(similarity_matrix, damping_factor=0.85, max_iter=100, tol=1e-4):
    n = similarity_matrix.shape[0]
    scores = np.ones(n) / n
    for _ in range(max_iter):
        prev_scores = scores.copy()
        for i in range(n):
            scores[i] = (1 - damping_factor) + damping_factor * np.sum(similarity_matrix[i] * prev_scores / np.sum(similarity_matrix[i]))
        if np.linalg.norm(scores - prev_scores) < tol:
            break
    return scores

# Apply TextRank to the similarity matrices for both questions and answers
dataset['question_scores'] = dataset['question_similarity_matrix'].apply(text_rank)
dataset['answer_scores'] = dataset['answer_similarity_matrix'].apply(text_rank)

dataset[['question_scores', 'answer_scores']].head()


<ipython-input-9-7789c2d7c9d1>:9: RuntimeWarning: invalid value encountered in divide
  scores[i] = (1 - damping_factor) + damping_factor * np.sum(similarity_matrix[i] * prev_scores / np.sum(similarity_matrix[i]))
<ipython-input-9-7789c2d7c9d1>:9: RuntimeWarning: invalid value encountered in divide
  scores[i] = (1 - damping_factor) + damping_factor * np.sum(similarity_matrix[i] * prev_scores / np.sum(similarity_matrix[i]))


,question_scores,answer_scores
0,"[0.9998559017305454, 0.9998559017305456, 0.999...","[0.9998559017305451, 0.9998559017305458, 0.999..."
1,"[0.9997905000298701, 0.99979050002987, 0.99979...","[0.9998168371689721, 0.9998168371689722, 0.999..."
2,"[0.9998365271733078, 0.9998365271733076, 0.999...","[0.9998365271733082, 0.999836527173308, 0.9998..."
3,"[0.999838545356353, 0.9998385453563535, 0.9998...","[0.999813021276659, 0.999813021276659, 0.99981..."
4,"[0.9998769696694945, 0.9998769696694949, 0.999...","[0.9998168371689726, 0.9998168371689727, 0.999..."


# Generating the Summary

In [ ]:
def generate_summary(original_sentences, scores, num_sentences=5):
    ranked_sentence_indices = np.argsort(scores)[::-1]

    summary = []
    for i in range(min(num_sentences, len(original_sentences))):
        summary.append(original_sentences[ranked_sentence_indices[i]])

    return " ".join(summary)

# Generate summaries for both questions and answers
dataset['Issue_Extracted'] = dataset.apply(lambda x: generate_summary(x['processed_question'][0], x['question_scores']), axis=1)
dataset['Solution_Extracted'] = dataset.apply(lambda x: generate_summary(x['processed_answer'][0], x['answer_scores']), axis=1)

# Display the summaries
#dataset[['question_summary', 'answer_summary']].head()


In [ ]:
#Character encoding
import chardet

def detect_and_fix_encoding(text):
    if not isinstance(text, str):
        raise ValueError("Input must be a string")

    try:
        # Detect the encoding
        result = chardet.detect(text.encode('utf-8', errors='ignore'))
        encoding = result['encoding']

        # Decode text with detected encoding, handle errors
        if encoding:
            return text.encode(encoding, errors='ignore').decode('utf-8', errors='replace')
        else:
            # Fallback if encoding detection fails
            return text
    except Exception as e:
        # Handle unexpected errors
        print(f"Error processing text: {e}")
        return text

# Apply the function to each element in the specified columns
dataset['Issue_Extracted'] = dataset['Issue_Extracted'].apply(detect_and_fix_encoding)
dataset['Solution_Extracted'] = dataset['Solution_Extracted'].apply(detect_and_fix_encoding)


In [ ]:
# Normalize unicode characters
def normalize_unicode(text):
    return unicodedata.normalize('NFKD', text)

# Apply normalize_unicode to each element in the specified columns
dataset['Issue_Extracted'] = dataset['Issue_Extracted'].apply(normalize_unicode)
dataset['Solution_Extracted'] = dataset['Solution_Extracted'].apply(normalize_unicode)

In [ ]:
# Apply contractions.fix to each element in the specified columns
dataset['Issue_Extracted'] = dataset['Issue_Extracted'].apply(lambda x: contractions.fix(x))
dataset['Solution_Extracted'] = dataset['Solution_Extracted'].apply(lambda x: contractions.fix(x))

In [ ]:
# Function to replace problematic characters
def replace_problematic_characters(text):
    replacements = {
        'â€™': "'",
        'â€œ': '"',
        'â€': '"',
        'â€TM': "'",
        'â€¦': " ",


        # Add more replacements if necessary
    }
    for old, new in replacements.items():
        text = text.replace(old, new)
    return text

In [ ]:
# Apply replace_problematic_characters to each element in the specified columns
dataset['Issue_Extracted'] = dataset['Issue_Extracted'].apply(replace_problematic_characters)
dataset['Solution_Extracted'] = dataset['Solution_Extracted'].apply(replace_problematic_characters)

In [ ]:
# Display the summaries
summaries = dataset[['Question_title', 'Issue_Extracted', 'Solution_Extracted']]
summaries.head()

,Question_title,Issue_Extracted,Solution_Extracted
0,Separation of Students and Users in NestJS Mic...,"Also, creating a student involves a step of cr...",I would think of a general rule to keep one re...
1,Flutter Clean Architecture,Regards I hope I explain my question clearly. ...,Data Layer (Repository Layer):\n\nThe data is ...
2,Correct .NET Architecture for long running asy...,I am storing the task details in a MS SQL Serv...,There is no easy and reliable way to do this w...
3,Architecture for white-label mobile apps with ...,"How can you ""assemble"" applications from modul...",Use dependency injection -> Let us call a piec...
4,Implementing Data Source Selection Logic in Cl...,RemoteData - loads and caches data from the AP...,"The logic does not belong in the Repository, i..."


In [ ]:
from colorama import Fore, Style
import textwrap

def format_summary(row, width=80):
    title = f"{Fore.RED}Title:{Fore.WHITE} {row['Question_title']}"
    question_summary = f"{Fore.GREEN}Issue_Extracted:{Fore.WHITE}\n{textwrap.fill(row['Issue_Extracted'], width=width)}"
    answer_summary = f"{Fore.BLUE}Solution_Extracted:{Fore.WHITE}\n{textwrap.fill(row['Solution_Extracted'], width=width)}"

    return f"{title}\n\n{question_summary}\n\n{answer_summary}\n{Fore.YELLOW}{'-'*width}{Fore.WHITE}\n"

# Apply the formatting function to the top 5 rows in the summaries DataFrame
formatted_summaries = summaries.head(6).apply(format_summary, width=80, axis=1)

# Display the formatted summaries
for summary in formatted_summaries:
    print(summary)


Title: Separation of Students and Users in NestJS Microservice architecture

Issue_Extracted:
Also, creating a student involves a step of creating a user as well. Considering
this, creating users, contacts, and addresses are part of the user folder, and
repository files and entity files related to these are stored in the user
folder. Right now it is a Monolith and not a microservice architecture, but I am
developing it in a way that tomorrow it will be easy to switch to Microservice.
So I am using a command/query approach for developing my RestAPIs. I am still
able to do stuff I need to do, I am just thinking, that tomorrow If I switch to
a microservices approach where the user and student will be different
microservices with different databases, I will not be able to inject the
repository and somehow need to call the Rest API for user or student to achieve
this.

Solution_Extracted:
I would think of a general rule to keep one repository, always, per CQRS
handler. Where does the "user"

# Save the summarized data to a new Excel file

In [ ]:
# Save the summarized data to a new Excel file
file_path = '/content/TextRank_Summaries.xlsx'

dataset.to_excel(file_path, index=False, engine='openpyxl')

In [ ]:
# Verify that the file has been saved correctly
saved_data = pd.read_excel(file_path)
print(saved_data[['Question_title','Issue_Extracted', 'Solution_Extracted']].head())

                                      Question_title  \
0  Separation of Students and Users in NestJS Mic...   
1                         Flutter Clean Architecture   
2  Correct .NET Architecture for long running asy...   
3  Architecture for white-label mobile apps with ...   
4  Implementing Data Source Selection Logic in Cl...   

                                     Issue_Extracted  \
0  Also, creating a student involves a step of cr...   
1  Regards I hope I explain my question clearly. ...   
2  I am storing the task details in a MS SQL Serv...   
3  How can you "assemble" applications from modul...   
4  RemoteData - loads and caches data from the AP...   

                                  Solution_Extracted  
0  I would think of a general rule to keep one re...  
1  Data Layer (Repository Layer):\n\nThe data is ...  
2  There is no easy and reliable way to do this w...  
3  Use dependency injection -> Let us call a piec...  
4  The logic does not belong in the Repository, i..